In [3]:
import torch
import ultralytics
import cv2
import os
import json
from ultralytics import YOLO  



In [11]:

# Define your classes here (same order as during YOLO training)
classes = ["pedestrians", "bikes", "bicyclists", "e-scooters", "e-scooterists"]

name_model="MicroVision-Chalmers-x-distorted-20250318"
# Load YOLO model
model = YOLO("algos/"+name_model+".pt")

video_name='GS010647'
# Open the video file
video_path = "Videos/" + video_name + ".mp4"
cap = cv2.VideoCapture(video_path)

# Get video frame dimensions and fps
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Create output directories if they don't exist
images_dir = "yolo/datasets_"+ video_name +"/one/images"
labels_dir = "yolo/datasets_"+ video_name +"/one/labels"
os.makedirs(images_dir, exist_ok=True)
os.makedirs(labels_dir, exist_ok=True)

# Create the classes.txt file
classes_file = "yolo/datasets_"+ video_name +"/one/classes.txt"
with open(classes_file, "w") as class_f:
    for cls in classes:
        class_f.write(f"{cls}\n")

print(f"Classes file created at: {classes_file}")

# Define the codec and create a VideoWriter object
output_path = "predictions/annotated_video_"+ video_name + '_'+name_model +".mp4"
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

frame_idx = 0  # Frame counter

# Process video frames
while cap.isOpened():
    success, frame = cap.read()
    if not success:
        break

    # Run YOLO detection
    results = model.track(frame, persist=True, imgsz=1280,conf=0.1)

    # Visualize results on the frame
    annotated_frame = results[0].plot()

    # Save the current frame as an image for Label Studio
    image_file = os.path.join(images_dir, f"frame_{frame_idx:04d}.jpg")
    cv2.imwrite(image_file, frame)

    # Save detections in YOLO format
    label_file = os.path.join(labels_dir, f"frame_{frame_idx:04d}.txt")
    with open(label_file, "w") as f:
        for r in results[0].boxes:
            cls = int(r.cls)
            x_center, y_center, w, h = r.xywhn[0]  # normalized (x_center, y_center, width, height)

            # Write to YOLO label file
            f.write(f"{cls} {x_center:.6f} {y_center:.6f} {w:.6f} {h:.6f}\n")

          
      

    # Write the annotated frame to the output video
    out.write(annotated_frame)

    # Display the annotated frame (optional, can comment out)
    # cv2.imshow("Annotated Frame", annotated_frame)

    # Break loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

    frame_idx += 1  # Increment frame counter


# Release resources
cap.release()
out.release()
cv2.destroyAllWindows()


Classes file created at: yolo/datasets_GS010647/one/classes.txt

0: 640x1280 12 persons, 1 bicycle, 1 cyclist, 811.2ms
Speed: 6.5ms preprocess, 811.2ms inference, 6.4ms postprocess per image at shape (1, 3, 640, 1280)

0: 640x1280 13 persons, 1 bicycle, 786.8ms
Speed: 4.1ms preprocess, 786.8ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 1280)

0: 640x1280 10 persons, 1 cyclist, 768.0ms
Speed: 3.4ms preprocess, 768.0ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 1280)

0: 640x1280 13 persons, 758.9ms
Speed: 3.3ms preprocess, 758.9ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 1280)

0: 640x1280 10 persons, 1 bicycle, 762.6ms
Speed: 3.1ms preprocess, 762.6ms inference, 0.3ms postprocess per image at shape (1, 3, 640, 1280)

0: 640x1280 9 persons, 2 cyclists, 742.5ms
Speed: 3.1ms preprocess, 742.5ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 1280)

0: 640x1280 10 persons, 1 bicycle, 711.6ms
Speed: 3.1ms preprocess, 711.6ms i

In [47]:
import yaml

# Définition du dataset
dataset = {
    "train": "/Users/martin.dejaeghere/Microsint2/yolo/datasets_GS010646",
    "val": "/Users/martin.dejaeghere/Microsint2/yolo/datasets_tolabel_copie",
    "nc": 5,  # Nombre de classes
    "names": ['pedestrians', 'bikes', 'bicyclists', 'e-scooterists', 'e-scooters'] # Classes
}

# Sauvegarde dans un fichier
with open("dataset.yaml", "w") as file:
    yaml.dump(dataset, file, default_flow_style=False)

print("Fichier dataset.yaml créé avec succès !")



Fichier dataset.yaml créé avec succès !


In [ ]:

# Validate with a custom dataset
name_model="MicroVision-Chalmers-x-distorted-20250318"
# Load YOLO model
model = YOLO("algos/"+name_model+".pt")

metrics = model.val(data="dataset.yaml",conf=0.3)  # output optional
print(metrics.box.map)  # map50-95

In [46]:
name_model="MicroVision-Chalmers-x-20250318"
model=YOLO("algos/"+name_model+".pt")

# Validate with a custom dataset
metrics = model.val(data="dataset.yaml",conf=0.4)  # output optional
print(metrics.box.map)  # map50-95


Ultralytics 8.3.87 🚀 Python-3.11.11 torch-2.6.0 CPU (Apple M3 Pro)
YOLO11x summary (fused): 190 layers, 56,832,799 parameters, 0 gradients, 194.4 GFLOPs


val: Scanning /Users/martin.dejaeghere/Microsint2/yolo/data_nimp/one/labels... 30 images, 0 backgrounds, 0 corrupt: 100%|██████████| 30/30 [00:00<00:00, 3136.79it/s]

val: New cache created: /Users/martin.dejaeghere/Microsint2/yolo/data_nimp/one/labels.cache



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [01:04<00:00, 32.07s/it]


                   all         30        754      0.261    0.00103    0.00199   0.000319
                person         30        726     0.0428    0.00413    0.00795    0.00127
               bicycle          8         14          0          0          0          0
               cyclist         11         12          0          0          0          0
          e-scooterist          2          2          1          0          0          0
Speed: 1.4ms preprocess, 2102.6ms inference, 0.0ms loss, 0.2ms postprocess per image
Results saved to runs/detect/val36
0.0003187212804116678


In [30]:
name_model="MicroVision-Chalmers-x-20250304"
model=YOLO("algos/"+name_model+".pt")

# Validate with a custom dataset
metrics = model.val(data="dataset.yaml",conf=0.4)  # output optional
print(metrics.box.map)  # map50-95


Ultralytics 8.3.87 🚀 Python-3.11.11 torch-2.6.0 CPU (Apple M3 Pro)
YOLO11x summary (fused): 190 layers, 56,832,799 parameters, 0 gradients, 194.4 GFLOPs


val: Scanning /Users/martin.dejaeghere/Microsint2/yolo/datasets_tolabel_copie/one/labels.cache... 46 images, 0 backgrounds, 0 corrupt: 100%|██████████| 46/46 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [01:55<00:00, 38.36s/it]


                   all         46        563      0.792      0.739      0.832      0.758
                person         46        420       0.95      0.583      0.777      0.706
               bicycle         23         67      0.939      0.687      0.829      0.731
               cyclist         21         28      0.714      0.714      0.804        0.8
             e-scooter         20         39      0.914      0.821      0.888      0.874
          e-scooterist          8          9      0.444      0.889      0.863       0.68
Speed: 1.3ms preprocess, 2468.9ms inference, 0.0ms loss, 0.4ms postprocess per image
Results saved to runs/detect/val31
0.758364834795386


In [44]:
# Définition du dataset
dataset = {
    "train": "/Users/martin.dejaeghere/Microsint2/yolo/datasets_GS010646",
    "val": "/Users/martin.dejaeghere/Microsint2/yolo/datasets_tolabel_copie",
    "nc": 2,  # Nombre de classes
    "names": ['persons', 'bikes'] # Classes
}

# Sauvegarde dans un fichier
with open("dataset.yaml", "w") as file:
    yaml.dump(dataset, file, default_flow_style=False)

print("Fichier dataset.yaml créé avec succès !")

name_model="yolo11x"
model=YOLO("algos/"+name_model+".pt")

# Validate with a custom dataset
metrics = model.val(data="dataset.yaml",conf=0.4,imgsz=1280)  # output optional
print(metrics.box.map)  # map50-95


Fichier dataset.yaml créé avec succès !
Ultralytics 8.3.87 🚀 Python-3.11.11 torch-2.6.0 CPU (Apple M3 Pro)
YOLO11x summary (fused): 190 layers, 56,919,424 parameters, 0 gradients, 194.9 GFLOPs


val: Scanning /Users/martin.dejaeghere/Microsint2/yolo/datasets_tolabel_copie/one/labels.cache... 46 images, 46 backgrounds, 0 corrupt: 100%|██████████| 92/92 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [04:29<00:00, 44.98s/it]


                   all         92        563      0.196      0.227      0.169       0.13
                person         46        420      0.544      0.614      0.451      0.335
               bicycle         23         67      0.438      0.522      0.393      0.316
                   car         21         28          0          0          0          0
            motorcycle         20         39          0          0          0          0
              airplane          8          9          0          0          0          0
Speed: 1.6ms preprocess, 2901.7ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to runs/detect/val35
0.13026452921773335


In [37]:
import os
import cv2
import shutil
import numpy as np

# Définition des chemins
dataset_path = "/Users/martin.dejaeghere/Microsint2/yolo/datasets_tolabel_copie/one"
images_path = os.path.join(dataset_path, "images")
labels_path = os.path.join(dataset_path, "labels")
labelling_path = os.path.join(dataset_path, "labelling")

# Création du dossier labelling s'il n'existe pas
os.makedirs(labelling_path, exist_ok=True)

# Définition des couleurs pour les classes
colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255), (255, 255, 0), (255, 0, 255)]

# Chargement des noms de classes
class_names = ['pedestrians', 'bikes', 'bicyclists', 'e-scooter', 'e-scooterist']

# Parcours des images
for image_name in os.listdir(images_path):
    if image_name.lower().endswith((".jpg", ".jpeg", ".png")):
        image_path = os.path.join(images_path, image_name)
        label_name = os.path.splitext(image_name)[0] + ".txt"
        label_path = os.path.join(labels_path, label_name)
        
        # Chargement de l'image
        image = cv2.imread(image_path)
        h, w, _ = image.shape
        
        # Vérification si l'annotation existe
        if os.path.exists(label_path):
            with open(label_path, "r") as file:
                lines = file.readlines()
                for line in lines:
                    parts = line.strip().split()
                    class_id = int(parts[0])
                    x_center, y_center, box_width, box_height = map(float, parts[1:])
                    
                    # Conversion des coordonnées YOLO en pixels
                    x1 = int((x_center - box_width / 2) * w)
                    y1 = int((y_center - box_height / 2) * h)
                    x2 = int((x_center + box_width / 2) * w)
                    y2 = int((y_center + box_height / 2) * h)
                    
                    # Dessiner la bounding box
                    color = colors[class_id % len(colors)]
                    cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)
                    cv2.putText(image, class_names[class_id], (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
            
            # Sauvegarde de l'image annotée
            output_path = os.path.join(labelling_path, image_name)
            cv2.imwrite(output_path, image)
        else:
            print(f"⚠️ Annotation manquante pour {image_name}, non traitée.")

